<a href="https://colab.research.google.com/github/datasith/ML-Notebooks-TensorFlow/blob/main/A_Gentle_Introduction_to_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Gentle Introduction to TensorFlow

In our previous TensorFlow [notebook](https://colab.research.google.com/github/datasith/ML-Notebooks-TensorFlow/blob/main/TensorFlow_Hello_World.ipynb), we learned about how to get started quickly with TensorFlow using Google Colab. In this tutorial, we are going to take a step back and review some of the basic components of building a deep learning model using TensorFlow. 

This will be a brief tutorial and will avoid using jargon and overcomplicated code. That said, this is perhaps the most basic of models you can build with TensorFlow. 

If fact, it is so basic that it's ideal for those starting to learn about TensorFlow and Deep Learning. So if you have a friend or colleague who wants to jump in, I highly encourage you to refer them to this tutorial as a starting point. Let's get started!

---
**Author:** Cisco Zabala ([@datasith](https://twitter.com/datasith) | [LinkedIn](https://www.linkedin.com/in/datasith/))

*Based on the work by Elvis Saravia ([Twitter](https://twitter.com/omarsar0) | [LinkedIn](https://www.linkedin.com/in/omarsar/)) on GitHub: [ML Notebooks](https://github.com/dair-ai/ML-Notebooks)*